# NPA hierarchy

The following code replicates the toy example from Pironio, S.; Navascués, M. & Acín, A. Convergent relaxations of polynomial optimization problems with noncommuting variables SIAM Journal on Optimization, SIAM, 2010, 20, 2157-2180.

## Defining a Polynomial Optimization Problem of Commuting Variables

Consider the following polynomial optimization problem:

$$\min_{x \in \R^2}{2x_1 x_2}$$

such that

$$-x_2^2 + x_2 + 0.5 \geq 0$$
$$x_1^2 - x_1 = 0.$$

The equality constraint is a simple projection. We either substitute it with two inequalities or treat the equality as a monomial substitution. The second option leads to a sparser SDP relaxation. The code samples below take this approach. In this case, the monomial basis is $\{1,x_1, x_2, x_1x_2, x_{22}\}$. The corresponding level-2 relaxation is written as

$$\min_{y}{2y_{12}}$$

such that

$$\begin{pmatrix}
1 & y_1 & y_2 & y_{12} & y_{22} \\
y_1 & y_1 & y_{12} & y_{12} & y_{122} \\
y_2 & y_{12} & y_{22} & y_{122} & y_{222} \\
y_{12} & y_{12} & y_{122} & y_{122} & y_{1222} \\
y_{22} & y_{122} & y_{222} & y_{1222} & y_{2222} \\
\end{pmatrix} \succeq 0$$

and

$$\begin{pmatrix}
-y_{22} + y_2 + 0.5 & -y_{122} + y_{12} + 0.5y_1 & -y_{222}+ y_{22} +0.5y_{2} \\
-y_{122} + y_{12} + 0.5y_1 & -y_{122} + y_{12} + 0.5y_1 & -y_{1222} + y_{122} + 0.5y_{12} \\
-y_{222} + y_{22} + 0.5y_{2} & -y_{1222} + y_{122} + 0.5y_{12} & -y_{2222} + y_{222} + 0.5y_{22} \\
\end{pmatrix} \succeq 0.$$

Apart from the matrices being symmetric, notice other regular patterns between the elements – these are recognized in the relaxation and the same SDP variables are used for matching moments. To generate the relaxation, first we set up a few helper variables, including the symbolic variables used to define the polynomial objective function and constraint. The symbolic manipulations are based on SymPy.

We start by importing the following libraries

In [1]:
from ncpol2sdpa import *

Then, we choose the level of the hierarchy and we generate variables

In [3]:
n_vars = 2 # Number of variables
level = 2  # Requested level of relaxation
x = generate_variables('x', n_vars)

By default, the generated variables are commutative. Alternatively, you can use standard SymPy symbols, but it is worth declaring them as real. With these variables, we can define the objective and the inequality constraint.

In [4]:
obj = x[0]*x[1] + x[1]*x[0] # objective function
inequalities = [-x[1]**2 + x[1] + 0.5] # constraints (\geq 0)

The equality, as discussed, is entered as a substitution rule:

In [5]:
substitutions = {x[0]**2 : x[0]}

### Generating and Solving the Relaxation

After we defined the problem, we need to initialize the SDP relaxation object with the variables, and request generating the relaxation given the constraints:

In [6]:
sdp = SdpRelaxation(x)
sdp.get_relaxation(level, objective=obj, inequalities=inequalities, substitutions=substitutions)

For large problems, getting the relaxation can take a long time. Once we have the relaxation, we can try to solve it. 

Currently three solvers are supported fully: SDPA, MOSEK, and CVXOPT. If any of them are available, we obtain the solution by calling the solve method. If you have multiple solvers available, you might want to specify which exactly you want to use (CVXOPT also requires PICOS on top of it). 

In [7]:
sdp.solve(solver='mosek')
print(sdp.primal, sdp.dual, sdp.status)

-0.7320508079547512 -0.7320508076316027 optimal


This gives a solution close to the optimum around -0.7321. The solution and some status information and the time it takes to solve it become part of the relaxation object.

If no solver is detected, or you want more control over the parameters of the solver, or you want to solve the problem in MATLAB, you export the relaxation to SDPA format:

In [14]:
sdp.write_to_file('example.dat-s')

### Analyzing the solution

We can study individual entries of the solution matrix by providing the monomial we are interested in. For example:

In [8]:
sdp[x[0]*x[1]]

np.float64(-0.36602540380169407)

The sums-of-square (SOS) decomposition is extracted from the dual solution:

In [9]:
sigma = sdp.get_sos_decomposition()

If we solve the SDP with the arbitrary-precision solver sdpa_gmp, we can find a rank loop at level two, indicating that convergence has been achieved.

In [ ]:
sdp.solve(solver='sdpa', solverparameters={"executable":"sdpa_gmp","paramsfile":"params.gmp.sdpa"})
sdp.find_solution_ranks()

The output for this problem is [2, 3], not showing a rank loop at this level of relaxation.

## Debugging the SDP relaxation

It often happens that solving a relaxation does not yield the expected results. To help understand what goes wrong, Ncpol2sdpa provides a function to write the relaxation in a comma separated file, in which the individual cells contain the respective monomials. The first line of the file is the objective function.

In [18]:
sdp.write_to_file("examples.csv")

Furthermore, the library can write out which SDP variable corresponds to which monomial by calling

In [19]:
sdp.save_monomial_index("monomials.txt")

## Defining and Solving an Optimization Problem of Noncommuting Variables

Consider a slight variation of the problem discussed in the previous sections: change the algebra of the variables from commutative to Hermitian noncommutative, and use the following objective function:

$$\min_{x \in \R^2}{x_1 x_2 + x_2 x_1}.$$

The constraints reamin identical:

$$-x_2^2 + x_2 +0.5 \geq 0$$
$$x_1^2 - x_1 = 0.$$

Defining the problem, generating the relaxation, and solving it follow a similar pattern, but we request operators instead of variables.

In [11]:
# Generate a vector of noncommuting variables
n_vars = 2
X = generate_operators('X', n_vars, hermitian=True)

# Define the objective, inequalities and substitutions
obj_nc = X[0] * X[1] + X[1] * X[0]
inequalities_nc = [-X[1] ** 2 + X[1] + 0.5]
substitutions_nc = {X[0]**2 : X[0]}

# Create the SDP relaxation
sdp_nc = SdpRelaxation(X)

# Get the relaxation
sdp_nc.get_relaxation(level, objective=obj_nc, inequalities=inequalities_nc,substitutions=substitutions_nc)

# Solve the SDP relaxation and print the results
sdp_nc.solve(solver='mosek')
print(sdp_nc.primal, sdp_nc.dual, sdp_nc.status)

-0.7499999993994902 -0.7499999936814759 optimal


This gives a solution very close to the analytical -3/4. Let us export the problem again:

In [32]:
sdp.write_to_file("examplenc.dat-s")

Solving this with the arbitrary-precision solver, we discover a rank loop:

In [ ]:
sdp.solve(solver='sdpa', solverparameters={"executable":"sdpa_gmp","paramsfile":"params.gmp.sdpa"})
sdp.find_solution_ranks()

The output is $[2, 2]$, indicating a rank loop and showing that the noncommutative case of the relaxation converges faster.